In [7]:
import os
import json
import requests

class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class llm_wrap(metaclass=Singleton):
    def __init__(self, config_file="ollama_config.json"):
        self.loaded = False
        self.config_data = self.load_config(config_file)
        self.load_response = self.load_model(self.config_data["LLM"])
        if self.load_response.status_code == 200: self.loaded = True

    def load_config(self, config_file):
        """
        Loads environment-variable-like keys from a JSON file and sets them.
        """
        with open(config_file, 'r') as f:
            config_data = json.load(f)
        for key, value in config_data.items():
            os.environ[key] = str(value)
        return config_data

    def load_model(self, model_name: str):
        """
        Instantiates an Ollama client for a given model.
        Because environment variables are already set,
        Ollama respects those concurrency/queue settings.
        """
        url = self.config_data["LOAD_MODEL_API_PATH"]
        payload = {
            "model": model_name,
            "stream": False  
        }

        response = requests.post(url, json=payload)
        print("Status code:", response.status_code)
        print("Response:", response.text)

        return response
    
    def process_prompt(self, prompt: str, chat = True):
        """
        Sends a prompt to the provided Ollama client and prints the 
        streamed output in real-time.
        """
        if self.loaded:
            url = self.config_data["CHAT_API_PATH"] if chat else self.config_data["GENERATE_API_PATH"]
            payload = {
                        "model": self.config_data["LLM"],
                        "messages": [
                            {
                            "role": "user",
                            "content": prompt
                            }
                        ],
                        "stream": True,
                        "options": {
                                    "temperature": self.config_data["TEMPERATURE"]
                                    }
                        }
            response = requests.post(url, json=payload, stream=True)
            self.complete_message = ""
            for line in response.iter_lines(decode_unicode=True):
                if line: 
                    try:
                        chunk = json.loads(line)
                        print(chunk)
                        if chunk["done"]:
                            return self.complete_message
                        else:
                            self.complete_message += chunk["message"]["content"]
                            yield self.complete_message
                    except json.JSONDecodeError as e:
                        print("Could not decode chunk:", line)
                        return False

            # return response
        else:
            print("Failed to load the model")


In [ ]:
for response in llm_wrap().process_prompt("think and say something"):
    # print(response)
    pass

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["How are you?", "Today is a great day", "I'm very hungry"])
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": bot_message})
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

In [1]:
import numpy as np
import gradio as gr

notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

def generate_tone(note, octave, duration):
    sr = 48000
    a4_freq, tones_from_a4 = 440, 12 * (octave - 4) + (note - 9)
    frequency = a4_freq * 2 ** (tones_from_a4 / 12)
    duration = int(duration)
    audio = np.linspace(0, duration, duration * sr)
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)
    return sr, audio

demo = gr.Interface(
    generate_tone,
    [
        # gr.Dropdown(notes, type="index"),
        # gr.Slider(4, 6, step=1),
        # gr.Textbox(value="1", label="Duration in seconds"),
        gr.Audio()
    ],
    "audio",
)
if __name__ == "__main__":
    demo.launch()

/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/gradio/utils.py:1024: UserWarning: Expected 3 arguments for function <function generate_tone at 0x7ba8a40c1f80>, received 1.
  warnings.warn(
/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/gradio/utils.py:1028: UserWarning: Expected at least 3 arguments for function <function generate_tone at 0x7ba8a40c1f80>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [1]:
import gradio as gr
from gradio import ChatMessage
from prompt_handler import handle_prompt
import gradio as gr
import json

class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(SingletonMeta, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class ChatInterface(metaclass=SingletonMeta):
    def __init__(self, chat_function):
        self.demo = gr.ChatInterface(
            chat_function,
            title="Verbal Communication Skills Trainer 💪",
            type="messages",
            multimodal=True,
            textbox=gr.MultimodalTextbox(sources=["microphone", "upload"]),
            # additional_inputs= gr.Audio()
        )
        

    def launch_interface(self):
        self.demo.launch()

# Load the content from a Markdown file
with open("improptu_speaking.md", "r", encoding="utf-8") as file:
    md_content = file.read()
md_content += "\n"
instructions = {
    "role": "user",
    "content": md_content,
    "metadata": None
}

def think_chat_function(message, history):
    print(history)
    print(type(history))
    if history: print(type(history[0]))
    # print("#######################")
    # print(message)
    
    # conv_stage = 0 if 'conv_stage' not in locals() or conv_stage>2 else conv_stage
    # lines = instructions["content"].splitlines()
    # lines.pop()
    # instructions["content"] = "\n".join(lines) + "\n" + str(conv_stage)
    history.append(instructions)
    # message = message["text"]
    del message["files"]
    message["content"] = message.pop("text")
    message["role"] = "user"
    history.append(message)
    response = [ChatMessage(
        content="",
        metadata={}
    )]

    think_stage = "think"
    llm_handler = handle_prompt()
    llm_generator = llm_handler.get_llm_output(history)
    if llm_generator:
        for llm_response in llm_generator:
            # print("response here###############")
            # print(llm_response)
            # print("response here###############")
            # llm_response_dict = json.loads(llm_response)
            # llm_response, conv_stage = llm_response_dict["content"], llm_response_dict["stage"]
            llm_response, think_stage = process_string(llm_response, think_stage)

            if think_stage=="think":
                response[-1].content = llm_response
                # response[-1].metadata = {"title": "Thinking", "id": conv_stage, "status": "pending"}
                response[-1].metadata = {"title": "Thinking", "status": "pending"}
            elif think_stage=="transition":
                response[-1].content = llm_response
                response[-1].metadata["status"] = "done"
                response[-1].metadata["thought_len"] = len(llm_response)
                response.append(ChatMessage(content=""))
            else:
                response[-1]=ChatMessage(content=llm_response[response[-2].metadata["thought_len"]:])
            yield response
        # print(response)
        # del llm_generator
        # yield response
        # conv_stage += 1
        if llm_handler.embedding and llm_handler.embedding not in llm_handler.cache: llm_handler.cache[llm_handler.embedding] = response[-1].content
    else: yield

def process_string(s, think_stage):
    if s.startswith("<think>") and "</think>" not in s: think_stage = "think"
    elif think_stage == "think": think_stage = "transition"
    else: think_stage = "respond"
    
    s = s.replace("<think>", "")
    s = s.replace("</think>", "")
    
    return s, think_stage


/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ChatInterface(think_chat_function).launch_interface()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[]
<class 'list'>
loading................
{'LLM': 'deepseek-r1:14b', 'TEMPERATURE': 0.2, 'EMBEDDING_MODEL': 'nomic-embed-text:latest', 'EMBED_TEMPERATURE': 0, 'OLLAMA_MAX_LOADED_MODELS': 1, 'OLLAMA_NUM_PARALLEL': 1, 'OLLAMA_MAX_QUEUE': 0, 'LOAD_MODEL_API_PATH': 'http://localhost:11434/api/pull', 'GENERATE_API_PATH': 'http://localhost:11434/api/generate', 'CHAT_API_PATH': 'http://localhost:11434/api/chat', 'EMBEDDING_API_PATH': 'http://localhost:11434/api/embed'}
Status code: 200
Response: {"status":"success"}
response###########################
<Response [200]>
[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': {'title': 'Thinking', 'status': 'done'}, 'content': 'Okay, so I\'m trying to figure out how to respond to the user\'s message where they said "hi." The context is that I\'m supposed to be a communication skills coach following a specific prompt template. Let me break this down step by step.\n\nFirst, I need to identify which 

In [4]:
print(history_out)

[]


In [ ]:
import gradio

with gradio.Blocks() as interface:
    recorder = gradio.Audio(sources='microphone', type='filepath', visible=False)
    action_btn = gradio.Button('Start')
    def next_line(action, _):
        if action == 'Start':
            return {action_btn: 'Next', recorder: gradio.update(visible=True)}
        else:
            return {action_btn: 'Done', recorder: gradio.update(visible=False)}
    action_btn.click(next_line, inputs=[action_btn, recorder], outputs=[action_btn, recorder])
interface.launch(share=True)

/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_10569/1725893084.py:30: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(elem_id="chatbot", bubble_full_width=False, type="messages")


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


{'text': '', 'files': []}


In [ ]:
import gradio as gr


def click_js():
    return """function audioRecord() {
    var xPathRes = document.evaluate ('//*[contains(@class, "record")]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null); 
    xPathRes.singleNodeValue.click();}"""


def action(btn):
    """Changes button text on click"""
    if btn == 'Speak': return 'Stop'
    else: return 'Speak'


def check_btn(btn):
    """Checks for correct button text before invoking transcribe()"""
    if btn != 'Speak': raise Exception('Recording...')


def transcribe():
    return 'Success'

with gr.Blocks() as demo:
    msg = gr.Textbox()
    audio_box = gr.Audio(label="Audio", sources="microphone", type="filepath", elem_id='audio')

    with gr.Row():
        audio_btn = gr.Button('Speak')
        clear = gr.Button("Clear")

    audio_btn.click(fn=action, inputs=audio_btn, outputs=audio_btn).\
              then(fn=lambda: None, js=click_js()).\
              then(fn=check_btn, inputs=audio_btn).\
              success(fn=transcribe, outputs=msg)

    clear.click(lambda: None, None, msg, queue=False)

demo.queue().launch(debug=True)

/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/gradio/blocks.py", line 2103, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Co

In [11]:
from pydantic import BaseModel
import json

class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]

print(Country.model_json_schema())
print(type(Country.model_json_schema()))
# Instead, provide actual data for a Country as JSON:
country_data_dict = {
    "name": "Japan",
    "capital": "Tokyo",
    "languages": ["Japanese"]
}

# Convert the dictionary to a JSON string
country_data_json = json.dumps(country_data_dict)

# Now validate and parse the JSON string into a Country object
country = Country.model_validate_json(country_data_json)
print(country, type(country))
print(country.dict(), type(country.dict()))

{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'capital': {'title': 'Capital', 'type': 'string'}, 'languages': {'items': {'type': 'string'}, 'title': 'Languages', 'type': 'array'}}, 'required': ['name', 'capital', 'languages'], 'title': 'Country', 'type': 'object'}
<class 'dict'>
name='Japan' capital='Tokyo' languages=['Japanese'] <class '__main__.Country'>
{'name': 'Japan', 'capital': 'Tokyo', 'languages': ['Japanese']} <class 'dict'>


/tmp/ipykernel_15517/3243008013.py:24: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(country.dict(), type(country.dict()))
